In [28]:
import pandas as pd
import numpy as np
import hashlib

In [3]:
df = pd.read_csv(r"C:\Development\ultimateNakMuay\data\raw\wiki_events_ufc.csv")
df.head()

,#,Event,Date,Venue,Location,Attendance,Ref.
0,648,UFC Fight Night: Dern vs. Hill,"May 20, 2023",UFC Apex,"Las Vegas, Nevada, U.S.",—,[23]
1,647,UFC on ABC: Rozenstruik vs. Almeida,"May 13, 2023",Spectrum Center,"Charlotte, North Carolina, U.S.","18,712",[24]
2,646,UFC 288: Sterling vs. Cejudo,"May 6, 2023",Prudential Center,"Newark, New Jersey, U.S.","17,559",[25]
3,645,UFC on ESPN: Song vs. Simón,"Apr 29, 2023",UFC Apex,"Las Vegas, Nevada, U.S.",—,[26]
4,644,UFC Fight Night: Pavlovich vs. Blaydes,"Apr 22, 2023",[27],NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           657 non-null    object
 1   Event       671 non-null    object
 2   Date        671 non-null    object
 3   Venue       671 non-null    object
 4   Location    611 non-null    object
 5   Attendance  577 non-null    object
 6   Ref.        513 non-null    object
dtypes: object(7)
memory usage: 36.8+ KB


In [5]:
df[df.isnull().any(axis=1)]

,#,Event,Date,Venue,Location,Attendance,Ref.
4,644,UFC Fight Night: Pavlovich vs. Blaydes,"Apr 22, 2023",[27],NaN,NaN,NaN
10,638,UFC 285: Jones vs. Gane,"Mar 4, 2023",T-Mobile Arena,"19,471",[33],NaN
11,637,UFC Fight Night: Muniz vs. Allen,"Feb 25, 2023",UFC Apex,—,[34],NaN
12,636,UFC Fight Night: Andrade vs. Blanchfield,"Feb 18, 2023",[35],NaN,NaN,NaN
17,631,UFC Fight Night: Cannonier vs. Strickland,"Dec 17, 2022",[40],NaN,NaN,NaN
...,...,...,...,...,...,...,...
666,NaN,UFC on ESPN: Strickland vs. Magomedov,"Jul 1, 2023",UFC Apex,[18],NaN,NaN
667,NaN,UFC on ABC: Emmett vs. Topuria,"Jun 24, 2023",VyStar Veterans Memorial Arena,"Jacksonville, Florida, U.S.",NaN,[19]
668,NaN,UFC on ESPN: Vettori vs. Cannonier,"Jun 17, 2023",UFC Apex,"Las Vegas, Nevada, U.S.",NaN,[20]
669,NaN,UFC 289: Nunes vs. Aldana,"Jun 10, 2023",Rogers Arena,"Vancouver, British Columbia, Canada",NaN,[21]


In [6]:
col_index = list(range(len(df.columns)-1, 1, -1))

In [8]:
df = pd.read_csv(r"C:\Development\ultimateNakMuay\data\raw\wiki_events_ufc.csv")

In [9]:
for i in df[df["Ref."].isna()].index:
    for j in  range(5, 1, -1):
        if not pd.isna(df.iloc[i,j]) and len(set(['[', ']']).intersection(list(df.iloc[i,j])))>0:
            df.iloc[i, 6] = df.iloc[i,j]
            df.iloc[i, j] = np.nan

In [10]:
for i in df[df["Attendance"].isna()].index:
    for j in  range(4, 1, -1):
        if not pd.isna(df.iloc[i,j]) and df.iloc[i,j].replace(',','').isnumeric():
            df.iloc[i, 5] = df.iloc[i,j]
            df.iloc[i, j] = np.nan

In [11]:
for i in df[~df["Date"].apply(lambda x: x[-4:].isnumeric())].index:
    df.loc[i,"Location"] = df.loc[i, "Venue"]
    df.loc[i,"Venue"] = df.loc[i, "Date"]
    df.loc[i,"Date"] = np.nan

In [12]:
cols2ffill = ["Date", "Venue", "Location", "Attendance"]
for col in cols2ffill:
    df[col] = df[col].fillna(method="ffill")

In [13]:
df['#'] = df['#'].apply(lambda x: "future" if pd.isna(x) else x if x.isnumeric() else "cancelled")

In [17]:
df["Attendance"] = df.apply(lambda row: np.nan if row['#'] == 'future' or row["Attendance"] == '—' else row['Attendance'], axis=1)

In [24]:
df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x))

In [26]:
df.drop(columns=["Ref."], axis=1, inplace=True)

In [29]:
cols2rename = {
    "#": "event_num",
}

cols2rename = {
    **{
        x: x.lower() for x in df.columns
    },
    **cols2rename
}

df.rename(columns=cols2rename, inplace= True)

In [32]:
df["event_id"] = df["event"].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

In [33]:
df

,event_num,event,date,venue,location,attendance,event_id
0,648,UFC Fight Night: Dern vs. Hill,2023-05-20,UFC Apex,"Las Vegas, Nevada, U.S.",NaN,d8efd546a501350524b7b0987463472ddc45083f561753...
1,647,UFC on ABC: Rozenstruik vs. Almeida,2023-05-13,Spectrum Center,"Charlotte, North Carolina, U.S.","18,712",b1ff2fc2110c5a0c1d8fd6fcda7d544ffb5a1fce513eb0...
2,646,UFC 288: Sterling vs. Cejudo,2023-05-06,Prudential Center,"Newark, New Jersey, U.S.","17,559",14ff8b34cf8ba961c51cbfae5e83e1870e2e9385014464...
3,645,UFC on ESPN: Song vs. Simón,2023-04-29,UFC Apex,"Las Vegas, Nevada, U.S.",NaN,1e43590228dd4bdfce337a7d45f67c32f78c9434eeeca7...
4,644,UFC Fight Night: Pavlovich vs. Blaydes,2023-04-22,UFC Apex,"Las Vegas, Nevada, U.S.",NaN,58cb1f65abe0d87d636582d45edbb984fe4c92044d67d3...
...,...,...,...,...,...,...,...
666,future,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,"Las Vegas, Nevada, U.S.",NaN,3864c7c9e2b76d47a7771d150726b4327c3fa88f05b4f6...
667,future,UFC on ABC: Emmett vs. Topuria,2023-06-24,VyStar Veterans Memorial Arena,"Jacksonville, Florida, U.S.",NaN,f796c71c34c1c47ec8d9ac26c5f015d2f0c36f4c1a6e1d...
668,future,UFC on ESPN: Vettori vs. Cannonier,2023-06-17,UFC Apex,"Las Vegas, Nevada, U.S.",NaN,b9d858650af3b4603d1f73dd9d46d337644d942d3b3c62...
669,future,UFC 289: Nunes vs. Aldana,2023-06-10,Rogers Arena,"Vancouver, British Columbia, Canada",NaN,a8b3628f1a194459323a285a9e55aaa74035164ad6967a...
